# Flatten gridded data for trend fitting

In [ ]:
# import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
import xarray as xr, regionmask, numpy as np

In [ ]:
ds_nm = "era5"
ndays = 29
varnm = "tn"


# load daily data
if ds_nm == "eobs":
    fnm = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/FS_heat/"+ds_nm+"_"+varnm+"_0.25deg.nc"
elif ds_nm == "era5":
    vnm = {"tn" : "tmin", "tx" : "tmax"}[varnm]
    fnm = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/FS_heat/"+ds_nm+"_"+vnm+"_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc"

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

varnm = varnm+str(ndays)+"x"

da = xr.open_dataset(fnm)
da = da[list(da.data_vars)[0]]

if "longitude" in da.dims: da = da.rename(longitude = "lon", latitude = "lat")

# mask land only
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
da = da.where(rm == 0)

# resample to annual
da = da.rolling(time = ndays).mean().resample(time = "AS").max()

# relabel dates as years
da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

# save the map for easier reconstruction later
da.mean("year").to_netcdf("map-tmplt_"+varnm+"_"+ds_nm+".nc")

# flatten & convert to data.frame
df = da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas()

# save data.frame as .csv (split into chunks if really large)
ncols = 2500
if df.shape[1] > ncols:
    for i in range(int(np.ceil(df.shape[1] / ncols))):
        df.iloc[:,slice(i*ncols,(i+1)*ncols)].to_csv("flattened/"+varnm+"-flattened_"+ds_nm+"_"+str(i+1).rjust(2,"0")+".csv")
else:
    df.to_csv("flattened/"+varnm+"-flattened_"+ds_nm+".csv")